In [72]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment
from azureml.pipeline.core import Pipeline, StepSequence
import datetime

In [73]:
# Environnement

compute_engine = 'jucalcul1'
cluster_name = 'jucalcul1'
dataset_name = 'trainingdataset'
experiment_name = "Template_hyperdrive"
subscription_id = "----------"
resource_group = "----------"
workspace_name = "deuxiemeespace"

# if not exist yet
#ws = Workspace.create(name='myworkspace',
#                      subscription_id='<azure-subscription-id>',
#                      resource_group='myresourcegroup',
#                      create_resource_group=True,
#                      location='eastus2')

ws = Workspace.get(subscription_id = subscription_id,
                   resource_group = resource_group,
                   name = workspace_name)

In [74]:
# Compute 
components_dir = '.'
os.makedirs(components_dir, exist_ok=True)


# Verify that the compute cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster : ', cluster_name)
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=3)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex, cluster_name)


Found existing cluster :  jucalcul1


In [75]:
# environment preparation : 
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
aml_run_config.target = compute_engine

USE_CURATED_ENV = False
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="fairnincluded") #AzureML-Tutorial
    aml_run_config.environment = curated_environment
    
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]','pandas','joblib','azureml-core==1.20.0','PyJWT==1.7.1','fairlearn',
        'azureml-defaults==1.32.0','azureml-train==1.32.0','azureml-contrib-fairness==1.32.0','interpret-community==0.18.1',
        "interpret-core==0.2.4",'azureml-interpret==1.32.0','azureml-datadrift==1.32.0','fairlearn==0.7.0','raiwidgets==0.9.4','facets-overview==1.0.0'], 
        pin_sdk_version=False)

In [76]:
# Step 0 environnement
from azureml.data.data_reference import DataReference
data_folder = os.path.join(os.getcwd(), 'data/')
os.makedirs(data_folder, exist_ok=True)
def_blob_store = Datastore.get(ws, 'saving')
ws.set_default_datastore('saving')

#sklearn_env = Environment.from_conda_specification(name = 'sklearn_envv2', file_path = './conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-Tutorial")

# Step 0 data ingestion 
raw_ds = Dataset.get_by_name(ws, name=dataset_name, version='latest') # training

# Create a Python environment for the pipeline experiment
#pipeline_environment = Environment.from_conda_specification(name = 'pipeline-env', file_path = './conda_dependencies.yml')
#pipeline_environment.python.user_managed_dependencies = False # Let Azure ML manage dependencies
#pipeline_environment.docker.enabled = True # Use a docker container
# Create a new runconfig object for the pipeline
#aml_run_config = RunConfiguration()
# Assign the environment to the run configuration
#aml_run_config.environment = pipeline_environment

#------------------------------------------------------------------------------------------------------------------------
# transformed data intermediary data
#train_datastore_path = OutputFileDatasetConfig(name="train_data", destination=(artifactstore, TRAIN_DIR)).as_upload()
#val_datastore_path = OutputFileDatasetConfig(name="val_data", destination=(artifactstore, VAL_DIR)).as_upload()
#test_datastore_path = OutputFileDatasetConfig(name="test_data", destination=(artifactstore, TEST_DIR)).as_upload()

#train_dataset = train_datastore_path.read_delimited_files().register_on_complete(name=AML_TRAIN_DATASET_NAME,
#                                                          description='Phishing Training Dataset',
#                                                          tags={'raw data version': raw_dataset.version,
#                                                                'pipeline version': PIPELINE_VERSION,
#                                                                'run version': RUN_VERSION})

#val_dataset = val_datastore_path.read_delimited_files().register_on_complete(name=AML_VAL_DATASET_NAME,
#                                                      description='Phishing Validation Dataset',
#                                                      tags={'raw data version': raw_dataset.version,
#                                                            'pipeline version': PIPELINE_VERSION,
#                                                            'run version': RUN_VERSION})

#test_dataset = test_datastore_path.read_delimited_files().register_on_complete(name=AML_TEST_DATASET_NAME,
#                                                        description='Phishing Testing Dataset',
#                                                        tags={'raw data version': raw_dataset.version,
#                                                              'pipeline version': PIPELINE_VERSION,
#                                                              'run version': RUN_VERSION})
# register clean data preprocess ---- 
#step1_output_ds = fileConfig.register_on_complete(name='processed_data', description = 'files from step1')

#------------------------------------------------------------------------------------------------------------------------



# specific path for object saving
fileConfig = OutputFileDatasetConfig(name='file_dataset', destination=(def_blob_store, "21-09-2021_template/last_structured/file")).as_mount()
fileConfig2 = OutputFileDatasetConfig(name='file_dataset2', destination=(def_blob_store, "21-09-2021_template/last_structured/file")).as_mount()
model_path = OutputFileDatasetConfig(name='model', destination=(def_blob_store, "21-09-2021_template/last_structured/model")).as_mount()
model_path2 = OutputFileDatasetConfig(name='model2', destination=(def_blob_store, "21-09-2021_template/last_structured/model")).as_mount()
metrics_path = OutputFileDatasetConfig(name='metrics', destination=(def_blob_store, "21-09-2021_template/last_structured/metrics")).as_mount()
dataset_param = PipelineData('dataset')  # provide output of one step and an input to the one more step further
datadir_param = PipelineData('datadir', is_directory=True)


# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = compute_engine,
                         runconfig=aml_run_config,
                         allow_reuse=True,
                         outputs=[dataset_param, datadir_param],
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'), # has_named_input is the method for tabular dataset 
                                      '--dataset', dataset_param, 
                                      '--datadir', datadir_param
                                      ])


# Step 2 : training 
train_src = ScriptRunConfig(source_directory = 'scripts',
                            script = 'model_script2.py',
                            compute_target = compute_engine,
                            arguments = ["--input-dir", fileConfig, # directory to save all objects
                                         '--dataset', dataset_param, #get data from previous step
                                         '--metrics-path', metrics_path,
                                         '--model-path',model_path  # model path
                                         ],
                            environment = sklearn_env)


# Step 2 : hyperparameter 
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy"),
    "--max_depth": choice(2, 8, 16)
    }
)

hd_config = HyperDriveConfig(run_config=train_src, 
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='precision', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4, # small nb for testing 
                             max_concurrent_runs=2)


    
hd_step = HyperDriveStep(
    allow_reuse=True,
    inputs=[dataset_param, datadir_param],
    name='hyperparameters',
    hyperdrive_config=hd_config)



# step 3 : register best model
register_model = PythonScriptStep(name = 'Model Registration & Evaluation',
                                  script_name = 'scripts/register_model2.py',
                                  compute_target = compute_engine,
                                  runconfig = aml_run_config,
                                  allow_reuse = True,
                                  arguments=[
                                          '--data_datastore_path', fileConfig2,
                                          '--model_path', model_path2
                                          ],
                                  )


# step 4 : publish pipeline
pipeline_training = [step1, hd_step, register_model]
pipeline = Pipeline(workspace=ws, steps =StepSequence(steps=pipeline_training))
pipeline_run = pipeline.submit(experiment_name, regenerate_outputs=True)

published_pipeline1 = pipeline.publish(
                        name="Template_Pipeline_Notebook",
                        description="Published Pipeline Description",
                        version="1.0")



# step 5 : schedule pipeline run every day
#recurrence = ScheduleRecurrence(frequency='Day', interval=1)
#recurring_schedule = Schedule.create(ws, name='DailySchedule', 
#                            description='Once a day',
#                            pipeline_id=published_pipeline1.id, 
#                            experiment_name= experiment_name, 
#                            recurrence=recurrence)

Created step prepare data [be6913af][c8f9d903-cec4-469a-89b0-ebd7f1518d6d], (This step will run and generate new outputs)
Created step hyperparameters [7c4b9fd4][9fb51f9d-a2a5-4ef1-b1dc-207563cc6966], (This step will run and generate new outputs)
Created step Model Registration & Evaluation [636876c0][104bc43d-b209-42a2-845f-108d2a4d431e], (This step will run and generate new outputs)
Submitted PipelineRun f290d8f7-7bdb-4bee-9302-da743cbb1e05
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f290d8f7-7bdb-4bee-9302-da743cbb1e05?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [77]:
# Model Output Arguments
run = Run.get_context()


for hyperdrive_run in pipeline_run.get_children(tags={"azureml.pipelineComponent" : "masterhyperdrivecloud"}):
    for train_run in hyperdrive_run.get_children():
        TRAIN_RUN_ID = train_run.get_details()["runId"]
        #BEST_CHILD_RUN_ID = train_run.get_details()['properties']
        #print(BEST_CHILD_RUN_ID)

In [78]:
pipeline_run.get_children(tags={"azureml.pipelineComponent" : "masterhyperdrivecloud"})

<generator object Run._rehydrate_runs at 0x0000020208728C80>

In [79]:
for child_run in pipeline_run.get_children():
    listid = child_run.id
    print('id parent', listid)
    for  child_run2 in child_run.get_children():
        print('id pipeline step',child_run2)
        for  child_run3 in child_run2.get_children():
            listid = child_run3.id
            summaryid.append(listid)
            #list_metric = child_run3.get_metrics()['Test Precison']
            #summarymetric.append(list_metric)

In [80]:
pipeline_run.get_details()['properties']
summaryid = [0]
summarymetric = [0]

for child_run in pipeline_run.get_children():
    listid = child_run.id
    print('id parent', listid)
    for  child_run2 in child_run.get_children():
        print('id pipeline step',child_run2)
        for  child_run3 in child_run2.get_children():
            listid = child_run3.id
            summaryid.append(listid)
            list_metric = child_run3.get_metrics()['Test Precison']
            summarymetric.append(list_metric)

print(summarymetric)
print(summaryid)
maxi = summarymetric.index(max(summarymetric))
maxi_id = summaryid[maxi]
print('best model id : ', maxi_id)
import pickle

run_context = Run.get_context()
print('start register model')
for child_run3 in child_run2.get_children():
    if child_run3.id == maxi_id:
        #child_run3.register_model(model_name='rf', model_path='outputs/model/saved_model.pkl')
        print("ok it's saved")
        #model = child_run3.upload_file('outputs/model/saved_model.pkl','outputs/model/saved_model.pkl')
        print(model)
       # model = pickle.load(open('outputs/model/saved_model.pkl', 'rb'))
        metrics = child_run3.get_file_names()
        print(metrics)
        #df = pd.DataFrame(list(metrics.items()),columns = ['metrics','value']) 
        print(df)

[0]
[0]
best model id :  0
start register model


NameError: name 'child_run2' is not defined

In [ ]:
for child_run3 in child_run2.get_children():
    if child_run3.id == maxi_id:
        #child_run3.register_model(model_name='rf_tuning', model_path='outputs_here/model.pkl')
        #print("ok it's saved")
        #print(child_run3.get_details())